# SQL - Dataset: theLook eCommerce

#### Fetching the data available on [Google Cloud](https://console.cloud.google.com/marketplace/product/bigquery-public-data/thelook-ecommerce?hl=es-419), through the BigQuery API.

In [1]:
from google.cloud import bigquery
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
# Create a "Client" object
client = bigquery.Client()
# Construct a reference to the dataset
dataset_ref = client.dataset("thelook_ecommerce", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# List all the tables in the dataset
tables = list(client.list_tables(dataset))
bigquery_tables = {}

# Print names of all tables in the dataset and makes a reference for each table
for table in tables:  
    table_name = table.table_id
    bigquery_tables[table_name] = f'`bigquery-public-data.thelook_ecommerce.{table_name}`'
    print(table_name)


Using Kaggle's public dataset BigQuery integration.
distribution_centers
events
inventory_items
order_items
orders
products
users


#### The functions below were created to ease the execution and show the size of the queries.

In [2]:
#Quick sketch for any given table
def sketch_table(table_name):
    # Construct a reference to the selected (first argument) table
    table_ref = dataset_ref.table(table_name)

    # API request - fetch the table
    table = client.get_table(table_ref)

    return client.list_rows(table, max_results=10).to_dataframe()
    #return table.schema

In [3]:
def show_query_size():
    # Create a QueryJobConfig object to estimate size of query without running it
    dry_run_config = bigquery.QueryJobConfig(dry_run=True)

    # API request - dry run query to estimate costs
    dry_run_query_job = client.query(query, job_config=dry_run_config)

    print("This query will process {} Megabytes.".format(dry_run_query_job.total_bytes_processed / 1000000))

In [4]:

def run_query():
    # Set up the query
    query_job = client.query(query)

    # API request - run the query, and return a pandas DataFrame
    query_df = query_job.to_dataframe()
    return query_df

# **Users Analysis**

#### Let's see how many costumers are registered, first and last date records of enrollment and average age

In [5]:
query = f"""
        SELECT COUNT(DISTINCT(id)) AS total_users, 
        MIN(EXTRACT(DATE FROM created_at)) AS first_customer,
        MAX(EXTRACT(DATE FROM created_at)) AS last_customer,
        AVG(age) AS average_age
        FROM {bigquery_tables['users']}
        """
show_query_size()
run_query()

This query will process 2.4 Megabytes.


,total_users,first_customer,last_customer,average_age
0,100000,2019-01-02,2022-07-20,41.01089


#### Sorted customer's total purchase value according to his user id and other data

In [6]:
query = f"""
        WITH orders_details AS (
        SELECT orders.order_id, orders.user_id, orders.num_of_item, order_items.sale_price
        FROM {bigquery_tables['orders']} AS orders 
        INNER JOIN {bigquery_tables['order_items']} AS order_items
        ON orders.order_id = order_items.order_id
        ) , user_total_sales AS (
        SELECT orders_details.user_id,
        SUM(sale_price) AS orders_total_value,
        COUNT(orders_details.order_id) AS num_orders
        FROM orders_details INNER JOIN {bigquery_tables['users']} AS users
        ON orders_details.user_id = users.id
        GROUP BY orders_details.user_id
        )

        SELECT CONCAT(users.first_name, ' ', users.last_name) AS user_full_name,
        users.age AS age,
        EXTRACT(DATE FROM users.created_at) AS enrolment_date,
        sales.num_orders,
        sales.orders_total_value
        FROM user_total_sales AS sales
        INNER JOIN {bigquery_tables['users']} AS users
        ON sales.user_id = users.id
        ORDER BY sales.orders_total_value DESC
        """
show_query_size()
run_query()

This query will process 8.89871 Megabytes.


,user_full_name,age,enrolment_date,num_orders,orders_total_value
0,Gregory Wood,55,2021-03-23,9,1624.610001
1,Hannah Melendez,48,2020-05-16,4,1598.990000
2,Kristi Mosley,38,2020-09-24,6,1488.960001
3,Michael Harris,20,2020-10-28,7,1485.489998
4,Maria Mora,68,2020-02-18,2,1398.000000
...,...,...,...,...,...
79989,Natasha Nguyen,54,2020-02-17,1,1.510000
79990,Barbara Harvey,37,2020-02-07,1,1.500000
79991,Alicia Fox,32,2019-07-17,1,1.500000
79992,Amanda Reed,68,2019-03-20,1,0.020000


#### Total sales by gender and country

In [7]:
query = f"""
        WITH orders_details AS (
        SELECT orders.order_id, orders.user_id, orders.num_of_item, order_items.sale_price
        FROM {bigquery_tables['orders']} AS orders 
        INNER JOIN {bigquery_tables['order_items']} AS order_items
        ON orders.order_id = order_items.order_id
        ) 
        
        SELECT users.country, 
        COUNT(CASE WHEN users.gender = 'F' THEN 1 ELSE NULL END) AS female_users,
        COUNT(CASE WHEN users.gender = 'M' THEN 1 ELSE NULL END) AS male_users,
        COUNT(*) AS total_users,
        SUM (CASE WHEN users.gender = 'F' THEN orders_details.sale_price ELSE NULL END) AS female_sales,
        SUM (CASE WHEN users.gender = 'M' THEN orders_details.sale_price ELSE NULL END) AS male_sales,
        SUM(orders_details.sale_price) AS total_sales
        FROM {bigquery_tables['users']} AS users INNER JOIN orders_details 
        ON orders_details.user_id = id
        GROUP BY users.country
        ORDER BY total_sales DESC
        """
show_query_size()
run_query()

This query will process 6.98119 Megabytes.


,country,female_users,male_users,total_users,female_sales,male_sales,total_sales
0,China,30713,30753,61466,1.726638e+06,1.939853e+06,3.666491e+06
1,United States,20330,20125,40455,1.129611e+06,1.279319e+06,2.408930e+06
2,Brasil,13243,13351,26594,7.315910e+05,8.414481e+05,1.573039e+06
3,South Korea,4753,4824,9577,2.561930e+05,3.065225e+05,5.627155e+05
4,France,4306,4153,8459,2.419381e+05,2.634972e+05,5.054353e+05
5,United Kingdom,4259,3991,8250,2.447549e+05,2.498424e+05,4.945973e+05
6,Germany,3628,3933,7561,2.094219e+05,2.436187e+05,4.530406e+05
7,Spain,3644,3488,7132,2.069624e+05,2.196962e+05,4.266585e+05
8,Japan,2343,2194,4537,1.334597e+05,1.428265e+05,2.762862e+05
9,Australia,2056,1941,3997,1.183115e+05,1.216821e+05,2.399936e+05


# **Sales Analysis**

#### Top profiting product categories

In [8]:
query = f"""
        WITH ordered_cost AS (
        SELECT order_items.product_id, order_items.sale_price, inv_items.cost
        FROM {bigquery_tables['order_items']} AS order_items
        INNER JOIN {bigquery_tables['inventory_items']} AS inv_items
        ON order_items.inventory_item_id = inv_items.id
        )
        SELECT products.category AS category,
        COUNT(*) AS total_products,
        SUM(order_items.cost) AS total_cost,
        SUM(products.retail_price) AS total_retail_price,
        SUM(order_items.sale_price) AS total_sale_price,
        SUM(order_items.sale_price) - SUM(order_items.cost) AS profit
        FROM ordered_cost AS order_items
        INNER JOIN {bigquery_tables['products']} AS products
        ON order_items.product_id= products.id
        GROUP BY category
        ORDER BY profit DESC
        """
show_query_size()
run_query()

This query will process 12.972313 Megabytes.


,category,total_products,total_cost,total_retail_price,total_sale_price,profit
0,Outerwear & Coats,9019,580555.792118,1.305369e+06,1.305369e+06,724813.676701
1,Jeans,12662,667415.877296,1.247765e+06,1.247765e+06,580348.744279
2,Sweaters,11072,400269.434241,8.302222e+05,8.302222e+05,429952.755766
3,Suits & Sport Coats,5161,267118.263097,6.654185e+05,6.654185e+05,398300.266071
4,Swim,11434,333209.048842,6.573291e+05,6.573291e+05,324120.051886
5,Fashion Hoodies & Sweatshirts,11619,328353.413790,6.312374e+05,6.312374e+05,302883.966259
6,Sleep & Lounge,11342,269204.273429,5.570228e+05,5.570228e+05,287818.557979
7,Shorts,11307,260011.948445,5.187088e+05,5.187088e+05,258696.823040
8,Accessories,9918,170288.376397,4.251689e+05,4.251689e+05,254880.533226
9,Active,8848,182093.093492,4.345263e+05,4.345263e+05,252433.206744


#### Top profiting products

In [9]:
query = f"""
        WITH ordered_cost AS (
        SELECT order_items.product_id, order_items.sale_price, inv_items.cost
        FROM {bigquery_tables['order_items']} AS order_items
        INNER JOIN {bigquery_tables['inventory_items']} AS inv_items
        ON order_items.inventory_item_id = inv_items.id
        )
        SELECT products.name AS product_name,
        COUNT(*) AS total_products,
        SUM(order_items.cost) AS total_cost,
        SUM(products.retail_price) AS total_retail_price,
        SUM(order_items.sale_price) AS total_sale_price,
        SUM(order_items.sale_price) - SUM(order_items.cost) AS profit
        FROM ordered_cost AS order_items
        INNER JOIN {bigquery_tables['products']} AS products
        ON order_items.product_id = products.id
        GROUP BY product_name
        ORDER BY profit DESC
        """
show_query_size()
run_query()

This query will process 14.086609 Megabytes.


,product_name,total_products,total_cost,total_retail_price,total_sale_price,profit
0,NIKE WOMEN'S PRO COMPRESSION SPORTS BRA *Outst...,19,8365.391995,17157.00,17157.00,8791.608005
1,The North Face Apex Bionic Soft Shell Jacket -...,16,6599.124027,14448.00,14448.00,7848.875973
2,The North Face Denali Down Womens Jacket 2013,13,5473.986013,11739.00,11739.00,6265.013987
3,The North Face Apex Bionic Mens Soft Shell Ski...,10,4198.950015,9030.00,9030.00,4831.049985
4,Quiksilver Men's Rockefeller Walkshort,11,5194.959000,9933.00,9933.00,4738.041000
...,...,...,...,...,...,...
27253,Allegra K Ladies Full Length Elastic Waist Ski...,1,4.406190,7.61,7.61,3.203810
27254,Wool Arctic Socks,2,3.599960,5.98,5.98,2.380040
27255,Classic Tear Drop Mirror Lens Aviator Sunglasses,2,1.290000,3.44,3.44,2.150000
27256,Set of 2 - Replacement Insert For Checkbook Wa...,4,0.709520,1.96,1.96,1.250480


#### Distribution Centers numbers on sold products

In [10]:
query = f"""
        WITH dist_centers AS (
        SELECT dists_centers_table.name, dists_centers_table.latitude, dists_centers_table. longitude, products.id, products.cost, products.category
        FROM {bigquery_tables['products']} AS products
        INNER JOIN {bigquery_tables['distribution_centers']} AS dists_centers_table
        ON products.distribution_center_id = dists_centers_table.id
        ) 
        SELECT dist_centers.name AS dist_name, 
        dist_centers.latitude AS dist_lat,
        dist_centers.longitude AS dist_lon,
        COUNT(*) AS products_sold, 
        SUM(dist_centers.cost) AS total_cost, 
        SUM(order_items.sale_price) AS total_sale,
        (SUM(order_items.sale_price) - SUM(dist_centers.cost)) AS profit
        FROM {bigquery_tables['order_items']} AS order_items
        INNER JOIN dist_centers
        ON order_items.product_id = dist_centers.id
        GROUP BY dist_name, dist_lat, dist_lon
        ORDER BY profit DESC
        """
show_query_size()
run_query()

This query will process 3.592425 Megabytes.


,dist_name,dist_lat,dist_lon,products_sold,total_cost,total_sale,profit
0,Houston TX,29.7604,-95.3698,22370,717623.640137,1.534064e+06,816440.439819
1,Memphis TN,35.1174,-89.9711,23890,664773.362638,1.398037e+06,733263.579677
2,Chicago IL,41.8369,-87.6847,24030,639165.017494,1.342336e+06,703171.334250
3,Mobile AL,30.6944,-88.0431,18223,601348.823604,1.228210e+06,626861.657898
4,Philadelphia PA,39.9500,-75.1667,16802,527853.905966,1.075380e+06,547525.775198
5,Port Authority of New York/New Jersey NY/NJ,40.6340,-73.7834,16595,460700.523790,9.534762e+05,492775.646743
6,Los Angeles CA,34.0500,-118.2500,17195,460544.883955,9.525513e+05,492006.438405
7,New Orleans LA,29.9500,-90.0667,12822,376746.316453,8.005387e+05,423792.394503
8,Savannah GA,32.0167,-81.1167,11955,397707.869057,8.112750e+05,413567.140827
9,Charleston SC,32.7833,-79.9333,16939,336456.826841,6.798173e+05,343360.512476


#### Average number of items and Estimated Time Delivery (ETD) per order by country

In [11]:
query = f"""
        SELECT users.country AS country, 
        AVG(orders.num_of_item) AS AVG_num_items, 
        AVG(DATE_DIFF(orders.delivered_at, orders.created_at, HOUR)) AS ETD_hours,
        FROM {bigquery_tables['orders']} AS orders
        INNER JOIN {bigquery_tables['users']} AS users
        ON orders.user_id = users.id
        WHERE orders.status = 'Complete'
        GROUP BY country
        ORDER BY ETD_hours 
        """
show_query_size()
run_query()

This query will process 6.413219 Megabytes.


,country,AVG_num_items,ETD_hours
0,Deutschland,2.000000,41.000000
1,Colombia,1.000000,76.000000
2,Poland,1.368421,90.315789
3,United States,1.431450,94.436560
4,Germany,1.462257,94.436576
5,Japan,1.420854,94.836683
6,Brasil,1.456561,94.929691
7,South Korea,1.420448,94.952208
8,Spain,1.446341,95.048780
9,United Kingdom,1.426275,95.074074
